In [10]:
import pandas as pd
import torch
import numpy as np
import torch.utils.data
import torch.nn as nn
from pandas import read_csv
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score


In [11]:
def load_file(filepath):
    dataframe = read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

data = load_file('wifi_localization.txt')

df = pd.DataFrame(data)


colname = df.columns[df.shape[1]-1]
x=df.drop(colname , axis=1).values
y=df[colname]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)


X_train = torch.from_numpy(X_train)
X_test=torch.from_numpy(X_test)

y_train=torch.Tensor(np.array(y_train))
y_test=torch.Tensor(np.array(y_test))



for i in range(0,4):
    y_train[y_train==i+1]=i
    y_test[y_test==i+1]=i
y_truee=y_test

In [12]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(7, 50)
        self.l2 = nn.Linear(50, 25)
        self.l3 = nn.Linear(25, 4)

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        return self.l3(x)


model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


In [13]:
counter=0

batch_size=50
lenth=len(X_train)/batch_size
l=int(lenth)
X_train=X_train.float()
y_train=y_train.long()


model.train()

def train(X_train,y_train):
    
    for i in range(0,l):

        X_t=X_train[(batch_size*i):(batch_size*i+batch_size)]
        y_t=y_train[(batch_size*i):(batch_size*i+batch_size)]    

        optimizer.zero_grad()
        output=model(X_t)
        loss = criterion(output, y_t)
        loss.backward()
        optimizer.step()



    
    


In [14]:



def test(X_test,y_test):
    counter=1
    test_loss=0

    X_test=X_test.float()
    y_test=y_test.long()

    model.eval()

    temp=torch.tensor([])
    y_true=y_truee.cpu().detach().numpy()


    output=model(X_test)

    q=F.softmax(output,dim=1)
    qa=F.softmax(q,dim=1)
    aba=qa.cpu().detach().numpy()
    y_pred=np.argmax(aba,axis=1)
    acc=accuracy_score(y_test,y_pred)
    recall=recall_score(y_test,y_pred,average='macro')
    prec=precision_score(y_test,y_pred,average='macro')
    f1=f1_score(y_test,y_pred,average='macro')
    print("Accuracy {} Recall {} precision {} F score {}".format (acc,recall,prec,f1))

        
for i in range(0,40):
    train(X_train,y_train)
test(X_test,y_test) 

Accuracy 0.9225 Recall 0.9229834657897141 precision 0.9359903381642513 F score 0.9248490494834662
